In [46]:
import warnings

import numpy as np
import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    model_built_data_paths,
    get_derived_csv_path,
)

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [47]:
asset_type = "building"
years_ahead = 2

metrics = {
    "median": "unit_price_median",
    "gmean": "unit_price_gmean",
}

dataset_paths = {
    "transactions": get_derived_csv_path(asset_type),
}

granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]

metric_key = "gmean"
metric = metrics[metric_key]
metric_pct_chg = metric + "_pct_chg"


In [35]:
dataset_key = "transactions"
core_path = dataset_paths[dataset_key]
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"]["municipality"]

dataset_name = f"sequence_{dataset_key}_{asset_type}_{metric_key}_{years_ahead}"
model_built_data_path = model_built_data_paths[dataset_name]

In [36]:
df = pd.read_csv(core_path)
df = df.assign(asset_type=asset_type)

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

In [37]:
df[df["area_code"] == 13101][["year", "area_code", "area", "count"]]

,year,area_code,area,count
183,2005,13101,Tokyo-to Chiyoda-ku,111.0
184,2006,13101,Tokyo-to Chiyoda-ku,165.0
185,2007,13101,Tokyo-to Chiyoda-ku,133.0
186,2008,13101,Tokyo-to Chiyoda-ku,122.0
187,2009,13101,Tokyo-to Chiyoda-ku,129.0
188,2010,13101,Tokyo-to Chiyoda-ku,157.0
189,2011,13101,Tokyo-to Chiyoda-ku,164.0
190,2012,13101,Tokyo-to Chiyoda-ku,196.0
191,2013,13101,Tokyo-to Chiyoda-ku,322.0
192,2014,13101,Tokyo-to Chiyoda-ku,236.0


In [38]:
df[df["area_code"] == 13101][["year", "area_code", "area", "count"]].round(3).to_csv("migration.csv", index=False)

In [39]:
# prepare metrics
df = df.sort_values(by=group_by_columns, ascending=True)
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(periods=years_ahead)
# df = df[~df[metric_pct_chg].isna()] # I don't want to drop the data from the first two years. I.e. 2007 should have 2005 and 2006 data in it's window

In [40]:
# prepare additional factors
df["count_growth"] = df.groupby(granularity_columns)["count"].pct_change()
df["yearly_price_growth"] = df.groupby(granularity_columns)[metric].pct_change()

for column in ["count", "total_traded_area", metric]:
    df[f"{column}_log"] = df[column].apply(lambda x: np.log10(1 + x))
    df[f"{column}_log_normalized_yearly"] = df.groupby("year")[f"{column}_log"].transform(
        lambda x: (x - x.mean()) / x.std()
    )

for column in ["count_growth", "yearly_price_growth", metric_pct_chg]: # metric_pct_chg_normalized_yearly will be the key metric
    df[f"{column}_normalized_yearly"] = df.groupby("year")[column].transform(
        lambda x: (x - x.mean()) / x.std()
    )

In [41]:
df["migrations_is_available"] = df["net_migration_ratio"].notnull().astype(int)
df["taxable_income_is_available"] = df["taxable_income"].notnull().astype(int)
df["total_tax_is_available"] = df["total_tax"].notnull().astype(int)
df["dwellings_is_available"] = df["new_dwellings"].notnull().astype(int)
df["metric_pct_chg_is_available"] = df[metric_pct_chg].notnull().astype(int)

In [42]:
log_normalize_columns = [
    metric,
    "count",
    "total_traded_area",
    "in_migrations",
    "out_migrations",
    "population",
    "taxpayer_count",
    "taxable_income",
    "taxable_income_per_taxpayer",
    "total_tax",
    "new_dwellings",
    "existing_dwellings",
]

normalize_columns = [
    metric_pct_chg,
    "count_growth",
    "yearly_price_growth",
    "total_tax_growth",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "taxpayer_count_growth",
]

maintain_columns = [
    "migrations_is_available",
    "taxable_income_is_available",
    "dwellings_is_available",
    "total_tax_is_available",
    "metric_pct_chg_is_available"
]

id_columns = ["area_code", "area", "year", "asset_type"]

feature_columns = (
    [f"{column}_log_normalized_yearly" for column in log_normalize_columns]
    + [f"{column}_normalized_yearly" for column in normalize_columns]
    + maintain_columns
)

final_columns = id_columns + normalize_columns + log_normalize_columns + feature_columns

# add ratios and growths if necessary

df = df[final_columns]

In [43]:
df.to_csv(model_built_data_path, index=False)

In [44]:
df[df[metric_pct_chg] > 2]

,area_code,area,year,asset_type,unit_price_gmean_pct_chg,count_growth,yearly_price_growth,total_tax_growth,taxable_income_growth,taxable_income_per_taxpayer_growth,net_migration_ratio,new_dwellings_ratio,taxpayer_count_growth,unit_price_gmean,count,total_traded_area,in_migrations,out_migrations,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,total_tax,new_dwellings,existing_dwellings,unit_price_gmean_log_normalized_yearly,count_log_normalized_yearly,total_traded_area_log_normalized_yearly,in_migrations_log_normalized_yearly,out_migrations_log_normalized_yearly,population_log_normalized_yearly,taxpayer_count_log_normalized_yearly,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log_normalized_yearly,total_tax_log_normalized_yearly,new_dwellings_log_normalized_yearly,existing_dwellings_log_normalized_yearly,unit_price_gmean_pct_chg_normalized_yearly,count_growth_normalized_yearly,yearly_price_growth_normalized_yearly,total_tax_growth_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,net_migration_ratio_normalized_yearly,new_dwellings_ratio_normalized_yearly,taxpayer_count_growth_normalized_yearly,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,metric_pct_chg_is_available


In [45]:
df.describe()

,area_code,year,unit_price_gmean_pct_chg,count_growth,yearly_price_growth,total_tax_growth,taxable_income_growth,taxable_income_per_taxpayer_growth,net_migration_ratio,new_dwellings_ratio,taxpayer_count_growth,unit_price_gmean,count,total_traded_area,in_migrations,out_migrations,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,total_tax,new_dwellings,existing_dwellings,unit_price_gmean_log_normalized_yearly,count_log_normalized_yearly,total_traded_area_log_normalized_yearly,in_migrations_log_normalized_yearly,out_migrations_log_normalized_yearly,population_log_normalized_yearly,taxpayer_count_log_normalized_yearly,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log_normalized_yearly,total_tax_log_normalized_yearly,new_dwellings_log_normalized_yearly,existing_dwellings_log_normalized_yearly,unit_price_gmean_pct_chg_normalized_yearly,count_growth_normalized_yearly,yearly_price_growth_normalized_yearly,total_tax_growth_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,net_migration_ratio_normalized_yearly,new_dwellings_ratio_normalized_yearly,taxpayer_count_growth_normalized_yearly,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,metric_pct_chg_is_available
count,5109.000000,5109.000000,4437.000000,4768.000000,4768.000000,5076.000000,4796.000000,4796.000000,5109.000000,4541.000000,4796.000000,5.109000e+03,5109.000000,5109.000000,5109.000000,5109.000000,5.109000e+03,4.796000e+03,4.796000e+03,4796.000000,5.086000e+03,4541.000000,4.541000e+03,5.109000e+03,5.109000e+03,5.109000e+03,5109.000000,5109.000000,5109.000000,4796.000000,4796.000000,4796.000000,5086.000000,4541.000000,4541.000000,4.437000e+03,4.768000e+03,4.768000e+03,5076.000000,4796.000000,4796.000000,5109.000000,4541.000000,4796.000000,5109.0,5109.000000,5109.000000,5109.000000,5109.000000
mean,19951.210804,2014.494422,0.039301,0.082342,0.024981,0.008147,0.007734,0.000188,0.000389,0.016365,0.007442,2.490549e+05,154.863966,9014.518497,12350.539832,12091.629658,2.806315e+05,1.274519e+05,4.541585e+08,3476.237026,4.539933e+07,2360.549659,1.393424e+05,-2.225227e-16,-2.225227e-17,1.223875e-16,1.241625,1.237928,1.224236,1.355974,1.403237,1.264167,1.324476,0.803948,0.852997,-1.441263e-17,-5.588371e-19,-3.725581e-19,0.079489,0.130701,-0.048623,0.483039,0.328570,0.344609,1.0,0.938736,0.888824,0.995498,0.868467
std,10154.182987,4.616607,0.182055,0.394982,0.170813,0.039121,0.031494,0.022681,0.005111,0.006224,0.016485,1.702666e+05,351.699096,19580.371772,21150.070179,20283.606130,3.812278e+05,1.736213e+05,6.620432e+08,797.378101,7.775866e+07,3634.089902,1.941700e+05,9.983346e-01,9.983346e-01,9.983346e-01,0.591985,0.611627,0.570587,0.572401,0.562659,1.096887,0.509518,0.769987,0.920275,9.983079e-01,9.983204e-01,9.983204e-01,0.770982,0.511721,0.537243,0.577482,0.822158,0.506083,0.0,0.239838,0.314381,0.066951,0.338015
min,1100.000000,2005.000000,-0.749036,-0.812500,-0.771530,-0.466623,-0.313802,-0.192218,-0.036352,0.001265,-0.337342,3.947562e+04,3.000000,100.000000,292.000000,443.000000,1.628800e+04,7.118000e+03,2.125598e+07,2441.662747,2.089891e+06,44.000000,1.578900e+04,-3.155350e+00,-2.078430e+00,-2.820555e+00,-0.669647,-0.485729,-0.308341,-0.211530,-0.169920,-0.755111,-0.179642,-1.878069,-0.860861,-4.982102e+00,-2.903888e+00,-4.150723e+00,-10.382247,-6.466902,-6.912359,-2.505351,-2.074150,-7.589756,1.0,0.000000,0.000000,0.000000,0.000000
25%,12222.000000,2011.000000,-0.064530,-0.125000,-0.063127,-0.008426,-0.002646,-0.007888,-0.002444,0.012358,0.000488,1.524865e+05,20.000000,1335.000000,3557.000000,3653.000000,1.014110e+05,4.464725e+04,1.442870e+08,3092.446050,1.409453e+07,691.000000,4.806900e+04,-6.547715e-01,-7.445491e-01,-7.409540e-01,0.848132,0.834077,0.852999,0.972518,1.003140,0.630617,1.038575,0.255703,0.197203,-5.379974e-01,-5.289933e-01,-5.186362e-01,-0.184934,-0.097563,-0.254136,0.145330,-0.187217,0.038677,1.0,1.000000,1.000000,1.00